# Migrating from Maple to q-Kangaroo

This guide maps functions from Frank Garvan's Maple packages (`qseries`, `thetaids`, `ETA`)
to their **q-Kangaroo** equivalents. Each section shows the Maple syntax alongside the
corresponding Python call, so researchers can translate existing Maple scripts directly.

q-Kangaroo covers all 13 function groups from Garvan's packages and adds extensions
(mock theta functions, algorithmic summation, nonterminating identity proofs) that have
no Maple equivalent.

## Quick Setup

Install q-Kangaroo and import everything:

```
pip install q-kangaroo
```

All q-Kangaroo functions take a `QSession` as the first argument (except standalone
utilities like `partition_count`, `sift`, and the algorithmic summation functions).

In [1]:
from q_kangaroo import *

s = QSession()

---

## Group 1: Pochhammer and q-Binomial

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `aqprod(a, q, n)` | `aqprod(s, num, den, pow, n, order)` | Monomial $a = \frac{\text{num}}{\text{den}} \cdot q^{\text{pow}}$ |
| `aqprod(a, q, infinity)` | `aqprod(s, num, den, pow, None, order)` | `n=None` for infinite product $(a;q)_\infty$ |
| `qbin(n, k, q)` | `qbin(s, n, k, order)` | q-binomial coefficient $\binom{n}{k}_q$ |

In [2]:
# Maple: aqprod(1, q, 3)  -->  (1;q)_3 = (1-1)(1-q)(1-q^2) = 0?
# No: (1;q)_3 = (1-q)(1-q^2)(1-q^3) when a=q^0=1 means base q^0
# Actually aqprod(s, 1, 1, 0, 3, 10) computes (1*q^0; q)_3 = (1;q)_3
finite = aqprod(s, 1, 1, 1, 3, 10)  # (q;q)_3 = (1-q)(1-q^2)(1-q^3)
print("(1;q)_3 =", finite)

# Maple: aqprod(a, q, infinity)  -->  infinite product
euler = aqprod(s, 1, 1, 1, None, 15)  # (q;q)_inf = Euler function
print("(q;q)_inf =", euler)

(1;q)_3 = 1 - q - q^2 + q^4 + q^5 - q^6 + O(q^10)
(q;q)_inf = 1 - q - q^2 + q^5 + q^7 - q^12 + O(q^15)


In [3]:
# Maple: qbin(5, 2, q)  -->  q-binomial coefficient
qbin(s, 5, 2, 15)

1 + q + 2*q^2 + 2*q^3 + 3*q^4 + 3*q^5 + 3*q^6 + 2*q^7 + 2*q^8 + q^9 + q^10 + O(q^15)

---

## Group 2: Named Products

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `etaq(d, t, q, N)` | `etaq(s, d, t, order)` | $\prod_{n \ge 0}(1 - q^{dn+t})$ |
| `jacprod(a, b, q, N)` | `jacprod(s, a, b, order)` | Jacobi triple product $J(a,b)$ |
| `tripleprod(z, b, t, q, N)` | `tripleprod(s, z_num, z_den, z_pow, b, t, order)` | Triple product with $z$ parameter |
| `quinprod(z, q, N)` | `quinprod(s, z_num, z_den, z_pow, order)` | Quintuple product |
| `winquist(a, b, q, N)` | `winquist(s, a_num, a_den, a_pow, b_num, b_den, b_pow, order)` | Winquist's 10-factor product |

In [4]:
# Maple: etaq(1, 1, q, 15)  -->  prod_{n>=0}(1 - q^{n+1}) = (q;q)_inf
print("etaq(1,1) =", etaq(s, 1, 1, 15))

# Maple: jacprod(1, 2, q, 20)  -->  J(1,2)
print("J(1,2) =", jacprod(s, 1, 2, 20))

etaq(1,1) = 1 - q - q^2 + q^5 + q^7 - q^12 + O(q^15)
J(1,2) = 1 - 2*q + 2*q^4 - 2*q^9 + 2*q^16 + O(q^20)


In [5]:
# Maple: tripleprod(-1, 2, 1, q, 20)  -->  theta_3 via triple product
# tripleprod with z=-1 (num=-1, den=1, pow=0), b=2, t=1
tripleprod(s, -1, 1, 0, 2, 1, 20)

1 - 2*q + 2*q^4 - 2*q^9 + 2*q^16 + O(q^20)

---

## Group 3: Theta Functions

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `theta2(q, N)` | `theta2(s, order)` | $\theta_2(q)$ in $q^{1/4}$ convention |
| `theta3(q, N)` | `theta3(s, order)` | $\theta_3(q) = \sum_{n=-\infty}^{\infty} q^{n^2}$ |
| `theta4(q, N)` | `theta4(s, order)` | $\theta_4(q) = \sum_{n=-\infty}^{\infty} (-1)^n q^{n^2}$ |

In [6]:
# Maple: theta2(q, 30), theta3(q, 20), theta4(q, 20)
print("theta2 =", theta2(s, 30))
print("theta3 =", theta3(s, 20))
print("theta4 =", theta4(s, 20))

theta2 = 2*q + 2*q^9 + 2*q^25 + O(q^30)
theta3 = 1 + 2*q + 2*q^4 + 2*q^9 + 2*q^16 + O(q^20)
theta4 = 1 - 2*q + 2*q^4 - 2*q^9 + 2*q^16 + O(q^20)


**Note:** `theta2` uses the $q^{1/4}$ convention. Exponents in the output
represent powers of $q^{1/4}$, so `2*q + 2*q^9 + 2*q^25` means
$2q^{1/4} + 2q^{9/4} + 2q^{25/4} + \cdots$

---

## Group 4: Partition Functions

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `numbpart(n)` | `partition_count(n)` | Exact $p(n)$ |
| `seq(numbpart(n), n=0..N)` | `partition_gf(s, order)` | Generating function |
| `rankgf(z, q, N)` | `rank_gf(s, z_num, z_den, order)` | Dyson rank GF |
| `crankgf(z, q, N)` | `crank_gf(s, z_num, z_den, order)` | Andrews-Garvan crank GF |
| *no Maple equiv* | `distinct_parts_gf(s, order)` | $\prod(1+q^k)$ |
| *no Maple equiv* | `odd_parts_gf(s, order)` | $\prod 1/(1-q^{2k+1})$ |

In [7]:
# Maple: numbpart(100)  -->  p(100) = 190569292
print("p(100) =", partition_count(100))

# Maple: series(1/mul(1-q^k,k=1..N), q, N)
print("partition_gf =", partition_gf(s, 10))

p(100) = 190569292
partition_gf = 1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + O(q^10)


In [8]:
# Maple: rankgf(1, q, 15) and crankgf(1, q, 15)
# At z=1, both reduce to the partition generating function
rgf = rank_gf(s, 1, 1, 15)
cgf = crank_gf(s, 1, 1, 15)
print("rank_gf(z=1) =", rgf)
print("crank_gf(z=1) =", cgf)
print("Both equal partition_gf:", repr(rgf) == repr(cgf))

rank_gf(z=1) = 1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + O(q^15)
crank_gf(z=1) = 1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + O(q^15)
Both equal partition_gf: True


In [9]:
# No direct Maple equivalent -- q-Kangaroo extension
dpgf = distinct_parts_gf(s, 15)
opgf = odd_parts_gf(s, 15)
print("Distinct parts:", dpgf)
print("Odd parts:     ", opgf)
print("Euler's theorem:", repr(dpgf) == repr(opgf))

Distinct parts: 1 + q + q^2 + 2*q^3 + 2*q^4 + 3*q^5 + 4*q^6 + 5*q^7 + 6*q^8 + 8*q^9 + 10*q^10 + 12*q^11 + 15*q^12 + 18*q^13 + 22*q^14 + O(q^15)
Odd parts:      1 + q + q^2 + 2*q^3 + 2*q^4 + 3*q^5 + 4*q^6 + 5*q^7 + 6*q^8 + 8*q^9 + 10*q^10 + 12*q^11 + 15*q^12 + 18*q^13 + 22*q^14 + O(q^15)
Euler's theorem: True


---

## Group 5: Utilities and Prodmake

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `sift(f, m, j)` | `sift(series, m, j)` | Extract $a_{mn+j}$ subsequence |
| `qfactor(f, q, N)` | `qfactor(series)` | Factor into $(1-q^i)$ |
| `prodmake(f, q, N)` | `prodmake(series, n)` | Andrews' product algorithm |
| `etamake(f, q, N)` | `etamake(series, n)` | Eta-quotient representation |
| `jacprodmake(f, q, N)` | `jacprodmake(series, n)` | Jacobi product representation |
| `mprodmake(f, q, N)` | `mprodmake(series, n)` | Modified product $(1+q^n)$ |
| *batch processing* | `s.batch_generate(func, params, order)` | Generate multiple series (QSession method) |

In [10]:
# Maple: sift(f, 5, 4)  -->  extract p(5n+4)
pgf50 = partition_gf(s, 50)
print("p(5n+4):", sift(pgf50, 5, 4))

p(5n+4): 5 + 30*q + 135*q^2 + 490*q^3 + 1575*q^4 + 4565*q^5 + 12310*q^6 + 31185*q^7 + 75175*q^8 + 173525*q^9 + O(q^10)


In [11]:
# Maple: prodmake(f, q, 10)  -->  find product exponents
pm = prodmake(pgf50, 10)
print("prodmake factors:", pm["factors"])

# Maple: etamake(f, q, 10)  -->  eta-quotient
em = etamake(pgf50, 10)
print("etamake factors:", em["factors"])
print("etamake q_shift:", em["q_shift"])

prodmake factors: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1}
etamake factors: {1: -1}
etamake q_shift: -1/24


In [12]:
# Maple: qfactor(f, q, N)  -->  factor into (1-q^i) factors
# Factor (q;q)_4 = (1-q)(1-q^2)(1-q^3)(1-q^4)
f4 = aqprod(s, 1, 1, 1, 4, 20)
print("qfactor:", qfactor(f4))

qfactor: {'scalar': Fraction(1, 1), 'factors': {1: 1, 2: 1, 3: 1, 4: 1}, 'is_exact': True}


---

## Group 6: Relation Discovery (Exact)

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `findlincombo(series_list, q, N)` | `findlincombo(target, candidates, topshift)` | $\text{target} = \sum c_i f_i$ |
| `findhom(series_list, d, q, N)` | `findhom(series_list, degree, topshift)` | Homogeneous poly relations |
| `findpoly(series_list, d, q, N)` | `findpoly(series_list, degree, topshift)` | General poly relations |
| `findnonhom(series_list, d, q, N)` | `findnonhom(series_list, degree, topshift)` | Non-homogeneous relations |
| `findprod(series, bases, q, N)` | `findprod(series_list, max_coeff, max_exp)` | Product representation |
| `findmaxind(series_list, q, N)` | `findmaxind(series_list, topshift)` | Maximal independent subset |

**Note:** In q-Kangaroo, `findlincombo(target, candidates, topshift)` finds
coefficients such that `target = sum(c_i * candidates[i])`. The `topshift`
parameter specifies how many leading coefficients to ignore.

In [13]:
# Discover Euler's theorem: distinct_parts_gf = odd_parts_gf
# findlincombo(target, candidates, topshift)
# target = distinct, candidates = [odd], should find coeff [1]
dpgf30 = distinct_parts_gf(s, 30)
opgf30 = odd_parts_gf(s, 30)

# Find: dpgf30 = c * opgf30
result = findlincombo(dpgf30, [opgf30], 0)
print("Euler's theorem: distinct = 1*odd + 0 =>", result)

Euler's theorem: distinct = 1*odd + 0 => [Fraction(1, 1)]


In [14]:
# Maple: findmaxind(series_list, q, N)
# Find maximal linearly independent subset
# partition_gf, odd_parts_gf, distinct_parts_gf: only 2 are independent
# (since odd = distinct by Euler's theorem)
pgf30 = partition_gf(s, 30)
ind = findmaxind([pgf30, opgf30, dpgf30], 0)
print("Maximal independent subset:", ind)

Maximal independent subset: [0, 1]


**Note:** `findprod` searches for a product representation using brute-force
enumeration and is computationally expensive for large bases.

---

## Group 7: Relation Discovery (Modular)

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `findcong(f, moduli, q, N)` | `findcong(series, moduli)` | Discover $a_{mn+j} \equiv 0 \pmod{d}$ |
| `findlincombo_modp(...)` | `findlincombomodp(target, candidates, p, topshift)` | Linear relations mod $p$ |
| `findhom_modp(...)` | `findhommodp(series_list, p, degree, topshift)` | Poly relations mod $p$ |

In [15]:
# Maple: findcong(f, [5,7,11], q, 100)  -->  Ramanujan congruences
pgf100 = partition_gf(s, 100)
congruences = findcong(pgf100, [5, 7, 11])

for c in congruences:
    print(f"p({c['modulus']}n + {c['residue']}) == 0 (mod {c['divisor']})")

p(5n + 4) == 0 (mod 5)
p(7n + 5) == 0 (mod 7)
p(11n + 6) == 0 (mod 11)


In [16]:
# findlincombomodp: find linear relation mod p
# target = distinct, candidate = [odd], mod 5
result = findlincombomodp(dpgf30, [opgf30], 5, 0)
print("findlincombomodp result:", result)

findlincombomodp result: [1]


---

## Group 8: Hypergeometric Series

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `qphihyper([a1,a2], [b1], q, z, N)` | `phi(s, [(n,d,p),...], [(n,d,p),...], z_n, z_d, z_p, order)` | ${}_r\phi_s$ |
| `qpsihyper([a1], [b1], q, z, N)` | `psi(s, [(n,d,p),...], [(n,d,p),...], z_n, z_d, z_p, order)` | ${}_r\psi_s$ bilateral |
| `qgauss(...)` | `try_summation(s, upper, lower, z_n, z_d, z_p, order)` | Try classical summation |
| `heine1(...)` | `heine1(s, upper, lower, z_n, z_d, z_p, order)` | Heine transform I |
| `heine2(...)` | `heine2(s, upper, lower, z_n, z_d, z_p, order)` | Heine transform II |
| `heine3(...)` | `heine3(s, upper, lower, z_n, z_d, z_p, order)` | Heine transform III |

Parameters are encoded as `(num, den, power)` tuples representing the monomial
$\frac{\text{num}}{\text{den}} \cdot q^{\text{power}}$.

In [17]:
# Maple: qphihyper([q, q^2], [q^4], q, q, 15)
# 2phi1(q, q^2; q^4; q, q)
phi(s, [(1,1,1), (1,1,2)], [(1,1,4)], 1, 1, 1, 15)

1 + q^3 + q^4 + q^6 + 2*q^7 + q^8 + 2*q^9 + 2*q^10 + 2*q^11 + 3*q^12 + 3*q^13 + 3*q^14 + O(q^15)

In [18]:
# Maple: try to evaluate via classical summation (q-Gauss, q-Vandermonde, etc.)
result = try_summation(s, [(1,1,1), (1,1,2)], [(1,1,4)], 1, 1, 1, 15)
print("Summation result:", result, "(no classical closed form for these parameters)")

Summation result: None (no classical closed form for these parameters)


In [19]:
# Maple: heine1(...)  -->  Heine's first transformation
# Returns (prefactor, result) where result = prefactor * transformed_series
prefactor, result = heine1(s, [(1,1,2),(1,1,3)], [(1,1,5)], 1, 1, 1, 15)
print("Heine I prefactor:", prefactor)
print("Heine I result:", result)

Heine I prefactor: 1 - q^2 - q^3 + q^4 + q^5 + q^7 - q^8 - q^9 + O(q^15)
Heine I result: 1 - q^2 - q^3 + q^4 + q^5 + q^7 - q^8 - q^9 + O(q^15)


---

## Group 9: Identity Proving (Eta)

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `proveid(lhs, rhs, level)` | `prove_eta_id(s, lhs_factors, rhs_factors, level)` | Valence formula proof |
| `search_id(...)` | `search_identities(query, search_type)` | Search identity database |

Eta-quotient factors are `(d, r)` tuples meaning $\eta(d\tau)^r$.

In [20]:
# Prove an eta identity: eta(tau)^2 / eta(2*tau) = eta(tau)^2 / eta(2*tau)
# LHS factors: [(1, 2), (2, -1)]  means eta(tau)^2 * eta(2*tau)^{-1}
# RHS factors: same (self-equality)
result = prove_eta_id(s, [(1, 2), (2, -1)], [(1, 2), (2, -1)], 2)
print("Status:", result["status"])
print("Level:", result["level"])
print("Sturm bound:", result["sturm_bound"])

Status: proved
Level: 2
Sturm bound: 1


---

## Group 10: Mock Theta, Appell-Lerch, and Bailey

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| *third order* | `mock_theta_f3(s, N)`, `mock_theta_phi3(s, N)`, `mock_theta_psi3(s, N)`, `mock_theta_chi3(s, N)`, `mock_theta_omega3(s, N)`, `mock_theta_nu3(s, N)`, `mock_theta_rho3(s, N)` | 7 third-order |
| *fifth order* | `mock_theta_f0_5(s, N)`, `mock_theta_phi0_5(s, N)`, `mock_theta_psi0_5(s, N)`, `mock_theta_chi0_5(s, N)`, `mock_theta_cap_f0_5(s, N)`, `mock_theta_f1_5(s, N)`, `mock_theta_phi1_5(s, N)`, `mock_theta_psi1_5(s, N)`, `mock_theta_chi1_5(s, N)`, `mock_theta_cap_f1_5(s, N)` | 10 fifth-order |
| *seventh order* | `mock_theta_cap_f0_7(s, N)`, `mock_theta_cap_f1_7(s, N)`, `mock_theta_cap_f2_7(s, N)` | 3 seventh-order |
| Appell-Lerch | `appell_lerch_m(s, a_pow, z_pow, order)` | Bilateral sum |
| Zwegers | `universal_mock_theta_g2(s, a_pow, order)`, `universal_mock_theta_g3(s, a_pow, order)` | Zwegers g-functions |
| Bailey lemma | `bailey_apply_lemma(s, pair_name, a, b, c, max_n, order)` | Apply Bailey's lemma |
| Bailey chain | `bailey_chain(s, pair_name, a, b, c, depth, max_n, order)` | Iterate lemma |
| Bailey discover | `bailey_discover(s, lhs, rhs, a, max_depth, order)` | Find pair match |

**Note:** Garvan's Maple packages do not include mock theta functions or Bailey
pair machinery. These are q-Kangaroo extensions.

In [21]:
# Third-order mock theta function f(q)
print("f(q) =", mock_theta_f3(s, 20))

# Seventh-order mock theta function F_0(q)
print("F0_7(q) =", mock_theta_cap_f0_7(s, 15))

f(q) = 1 + q + q^2 + q^3 + 2*q^4 + 2*q^5 + 3*q^6 + 3*q^7 + 4*q^8 + 5*q^9 + 6*q^10 + 7*q^11 + 9*q^12 + 10*q^13 + 13*q^14 + 15*q^15 + 18*q^16 + 21*q^17 + 26*q^18 + 30*q^19 + O(q^20)
F0_7(q) = 1 + q + q^2 + q^3 + q^4 + q^5 + 2*q^6 + 2*q^7 + 3*q^8 + 3*q^9 + 4*q^10 + 4*q^11 + 5*q^12 + 6*q^13 + 7*q^14 + O(q^15)


In [22]:
# Appell-Lerch bilateral sum m(a, z, q)
appell_lerch_m(s, 1, 1, 20)

1 + q + 2*q^2 + 2*q^3 + 4*q^4 + 4*q^5 + 7*q^6 + 8*q^7 + 12*q^8 + 14*q^9 + 20*q^10 + 24*q^11 + 34*q^12 + 40*q^13 + 54*q^14 + 66*q^15 + 86*q^16 + 104*q^17 + 136*q^18 + 164*q^19 + O(q^20)

In [23]:
# Bailey chain from Rogers-Ramanujan pair
# bailey_chain(s, pair_name, a, b, c, depth, max_n, order)
chain = bailey_chain(s, "rogers-ramanujan",
                     (1,1,0), (1,1,1), (1,1,2),  # a=1, b=q, c=q^2
                     2, 10, 20)
print("Chain length:", len(chain))

Chain length: 2


The 20 classical mock theta functions are named by order and variant with the
`mock_theta_` prefix. The Appell-Lerch and Bailey machinery provides the modern
framework for understanding mock theta functions (Zwegers, 2002).

---

## Group 11: q-Gosper Algorithm

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| *no direct equiv* | `q_gosper(upper, lower, z_n, z_d, z_p, q_n, q_d)` | Indefinite q-hypergeometric sum |

The q-Gosper algorithm decides whether a q-hypergeometric term has a
q-hypergeometric anti-difference (indefinite sum). Parameters do **not**
require a QSession.

In [24]:
# q-Gosper: indefinite summation
result = q_gosper([(1,1,-3), (1,1,2)], [(1,1,3)], 1, 1, 4, 2, 1)
print("Summable:", result["summable"])

Summable: True


---

## Group 12: Algorithmic Summation

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| `qZeil(...)` | `q_zeilberger(upper, lower, z_n, z_d, z_p, n, q_n, q_d, max_order)` | Creative telescoping |
| *WZ verify* | `verify_wz(upper, lower, z_n, z_d, z_p, n, q_n, q_d, max_order, max_k)` | Certificate verification |
| `qPetkovsek(...)` | `q_petkovsek(coefficients, q_num, q_den)` | Solve recurrence |

These functions take concrete values of $q$ (not symbolic) and operate on
q-hypergeometric series defined by parameter tuples.

In [25]:
# Full pipeline: q-Zeilberger on q-Vandermonde sum
# 2phi1(q^{-5}, q^2; q^3; q, q^4) at n=5, q=2
zeil_result = q_zeilberger(
    [(1,1,-5), (1,1,2)], [(1,1,3)],  # upper, lower
    1, 1, 4,  # z = q^4
    5,        # n = 5
    2, 1,     # q = 2
    3         # max_order
)
print("Recurrence found:", zeil_result["found"])
print("Order:", zeil_result["order"])
print("Coefficients:", len(zeil_result["coefficients"]))

Recurrence found: True
Order: 1
Coefficients: 2


In [26]:
# Verify the WZ certificate independently
wz_result = verify_wz(
    [(1,1,-5), (1,1,2)], [(1,1,3)],
    1, 1, 4,
    5,
    2, 1,
    3, 20  # max_order=3, max_k=20 evaluation points
)
print("WZ verified:", wz_result["verified"])
print("Recurrence order:", wz_result["order"])

WZ verified: True
Recurrence order: 1


---

## Group 13: Extensions

| Maple | q-Kangaroo | Notes |
|-------|------------|-------|
| *no equiv* | `prove_nonterminating(upper_fixed, n_offset, lower, z_offset, rhs_n, rhs_d, q_n, q_d, n_test, max_order)` | Chen-Hou-Mu proof |
| *no equiv* | `find_transformation_chain(s, src_upper, src_lower, src_z, tgt_upper, tgt_lower, tgt_z, max_depth, order)` | BFS transformation search |

In [27]:
# Prove nonterminating q-Gauss identity via Chen-Hou-Mu specialization
# LHS: 2phi1(q, q^2; q^3; q, q^{2+n}) with n_param = q^{2-n}
# RHS: (q^2;q)_n * (q;q)_n / ((q^3;q)_n * (q^0;q)_n)
nt_result = prove_nonterminating(
    [(1,1,1)],    # upper_fixed: [q]
    2,            # n_param_offset: q^{2-n}
    [(1,1,3)],    # lower: [q^3]
    2,            # z_pow_offset: z = q^{2+n}
    [2, 1],       # rhs_numer_bases: (q^2;q)_n * (q;q)_n
    [3, 0],       # rhs_denom_bases: (q^3;q)_n * (1;q)_n
    2, 1,         # q = 2
    5,            # n_test = 5
    3             # max_order = 3
)
print("Proved:", nt_result["proved"])
print("Recurrence order:", nt_result["recurrence_order"])

Proved: True
Recurrence order: 1


In [28]:
# Find transformation chain between two 2phi1 series
# Source: 2phi1(q^2, q^3; q^5; q, q)
# Apply Heine I: 2phi1(a,b;c;q,z) -> pf * 2phi1(c/b, z; az; q, b)
# Target: 2phi1(q^2, q; q^4; q, q^3)  [Heine I with a=q^2, b=q^3, c=q^5, z=q]
chain_result = find_transformation_chain(
    s,
    [(1,1,2), (1,1,3)], [(1,1,5)], 1, 1, 1,   # source
    [(1,1,2), (1,1,1)], [(1,1,4)], 1, 1, 3,   # target
    3, 30                                       # max_depth, order
)
print("Chain found:", chain_result["found"])
if chain_result["found"]:
    print("Steps:", len(chain_result["steps"]))
    print("Transformation:", chain_result["steps"][0]["name"])

Chain found: True
Steps: 1
Transformation: heine1


---

## Quick Reference: Complete Maple to q-Kangaroo Mapping

| # | Group | Maple | q-Kangaroo |
|---|-------|-------|------------|
| 1 | Pochhammer | `aqprod(a, q, n)` | `aqprod(s, num, den, pow, n, order)` |
| 2 | Pochhammer | `aqprod(a, q, infinity)` | `aqprod(s, num, den, pow, None, order)` |
| 3 | Pochhammer | `qbin(n, k, q)` | `qbin(s, n, k, order)` |
| 4 | Products | `etaq(d, t, q, N)` | `etaq(s, d, t, order)` |
| 5 | Products | `jacprod(a, b, q, N)` | `jacprod(s, a, b, order)` |
| 6 | Products | `tripleprod(z, b, t, q, N)` | `tripleprod(s, z_n, z_d, z_p, b, t, order)` |
| 7 | Products | `quinprod(z, q, N)` | `quinprod(s, z_n, z_d, z_p, order)` |
| 8 | Products | `winquist(a, b, q, N)` | `winquist(s, a_n, a_d, a_p, b_n, b_d, b_p, order)` |
| 9 | Theta | `theta2(q, N)` | `theta2(s, order)` |
| 10 | Theta | `theta3(q, N)` | `theta3(s, order)` |
| 11 | Theta | `theta4(q, N)` | `theta4(s, order)` |
| 12 | Partitions | `numbpart(n)` | `partition_count(n)` |
| 13 | Partitions | `seq(numbpart(n), ...)` | `partition_gf(s, order)` |
| 14 | Partitions | `rankgf(z, q, N)` | `rank_gf(s, z_num, z_den, order)` |
| 15 | Partitions | `crankgf(z, q, N)` | `crank_gf(s, z_num, z_den, order)` |
| 16 | Utilities | `sift(f, m, j)` | `sift(series, m, j)` |
| 17 | Utilities | `qfactor(f, q, N)` | `qfactor(series)` |
| 18 | Utilities | `prodmake(f, q, N)` | `prodmake(series, n)` |
| 19 | Utilities | `etamake(f, q, N)` | `etamake(series, n)` |
| 20 | Utilities | `mprodmake(f, q, N)` | `mprodmake(series, n)` |
| 21 | Relations | `findlincombo(...)` | `findlincombo(target, candidates, topshift)` |
| 22 | Relations | `findhom(...)` | `findhom(series_list, degree, topshift)` |
| 23 | Relations | `findpoly(...)` | `findpoly(series_list, degree, topshift)` |
| 24 | Relations | `findnonhom(...)` | `findnonhom(series_list, degree, topshift)` |
| 25 | Relations | `findprod(...)` | `findprod(series_list, max_coeff, max_exp)` |
| 26 | Relations | `findmaxind(...)` | `findmaxind(series_list, topshift)` |
| 27 | Modular | `findcong(...)` | `findcong(series, moduli)` |
| 28 | Modular | `findlincombo_modp(...)` | `findlincombomodp(target, candidates, p, topshift)` |
| 29 | Modular | `findhom_modp(...)` | `findhommodp(series_list, p, degree, topshift)` |
| 30 | Hypergeometric | `qphihyper(...)` | `phi(s, upper, lower, z_n, z_d, z_p, order)` |
| 31 | Hypergeometric | `qpsihyper(...)` | `psi(s, upper, lower, z_n, z_d, z_p, order)` |
| 32 | Hypergeometric | `qgauss(...)` | `try_summation(s, upper, lower, z_n, z_d, z_p, order)` |
| 33 | Hypergeometric | `heine1(...)` | `heine1(s, upper, lower, z_n, z_d, z_p, order)` |
| 34 | Identity | `proveid(...)` | `prove_eta_id(s, lhs_factors, rhs_factors, level)` |
| 35 | *Extension* | -- | `mock_theta_f3(s, N)` (20 functions total) |
| 36 | *Extension* | -- | `appell_lerch_m(s, a_pow, z_pow, order)` |
| 37 | *Extension* | -- | `bailey_chain(s, pair, a, b, c, depth, max_n, order)` |
| 38 | *Extension* | -- | `q_gosper(upper, lower, z_n, z_d, z_p, q_n, q_d)` |
| 39 | *Extension* | -- | `q_zeilberger(upper, lower, z, n, q, max_order)` |
| 40 | *Extension* | -- | `verify_wz(upper, lower, z, n, q, max_order, max_k)` |
| 41 | *Extension* | -- | `prove_nonterminating(...)` |
| 42 | *Extension* | -- | `find_transformation_chain(s, ...)` |

## Common Patterns

1. **Session argument:** All q-series functions take a `QSession` as the first argument
   (except `partition_count`, `sift`, `qfactor`, and the algorithmic functions
   `q_gosper`, `q_zeilberger`, `verify_wz`, `q_petkovsek`, `prove_nonterminating`).

2. **Monomial encoding:** Parameters like $a = \frac{3}{2} q^5$ are encoded as
   `(num, den, power)` tuples: `(3, 2, 5)`. The constant $1$ is `(1, 1, 0)`.

3. **Infinite products:** Use `n=None` in `aqprod` for $(a;q)_\infty$.

4. **Return types:** Most functions return `QSeries`. Analysis functions
   (`prodmake`, `etamake`, `qfactor`, `findcong`) return Python dicts.
   Relation discovery (`findlincombo`, etc.) returns lists of coefficients.
   Heine transforms return `(prefactor, result)` tuples.

5. **Batch generation:** `s.batch_generate(func_name, params, order)` is a
   `QSession` method for systematic parameter scans over generator functions.